In [84]:
# download to local
!dx download --lightweight -fr data

FileNotFoundError: [Errno 2] No such file or directory


In [16]:
#get list for files have not been download successfully
#df =  pd.read_csv("file_paths.txt", header=None)
result_file_path = os.path.join('/Users/xiaoqianxiao/UKB/data/derivatives', 'output.txt')
df =  pd.read_csv("file_paths.txt")
df['filename'] = df['filepath'].str.split(r'[/]', expand=True)[5]
directory = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
file_names = os.listdir(directory)
df['done_files'] = df['filename'].apply(lambda x: (x in file_names)).replace((1,0),('True','False'))
df.loc[df['done_files'] == False]['filepath'].to_csv('file_paths_remain.txt', sep='\t', index=False, header=False)
df.loc[df['done_files'] == False]['filename'].to_csv(result_file_path, sep='\t', index=False, header=False)

NameError: name 'pd' is not defined

In [17]:
import os
import zipfile
def get_all_folders(path):
    return {entry.name: entry.path for entry in os.scandir(path) if entry.is_dir()}
sourceData_path='/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
dict_data_sets = get_all_folders(sourceData_path)
def unzip_files(source_path, unzip_path):
    # Change to the download directory
    os.chdir(download_path)
    
    # Iterate over all .zip files in the directory
    for f in os.listdir(download_path):
        if f.endswith('.zip'):
            # Create a directory for the unzipped contents
            destination = os.path.join(timeseries_path, f[:-4])  # Remove ".zip" from filename
            os.makedirs(destination, exist_ok=True)
            
            # Unzip the file into the destination directory
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(destination)
            print(f"Extracted {f} to {destination}")
for folder in dict_data_sets.keys():
    download_path = dict_data_sets[folder]
    unzip_path = os.path.join('/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB',folder)
    unzip_files(download_path, unzip_path)

In [85]:
def unzip_files(source_path, unzip_path):
    # Change to the download directory
    os.chdir(download_path)
    # Iterate over all .zip files in the directory
    for f in os.listdir(download_path):
        if f.endswith('.zip'):
            # Create a directory for the unzipped contents
            destination = os.path.join(unzip_path, f[:-4])  # Remove ".zip" from filename
            os.makedirs(destination, exist_ok=True)
            
            # Unzip the file into the destination directory
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(destination)
            print(f"Extracted {f} to {destination}")
            
download_path = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
unzip_path = '/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB'
unzip_files(download_path, unzip_path)

Extracted 2284079_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2284079_31019_2_0
Extracted 4943295_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/4943295_31019_2_0
Extracted 3696737_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3696737_31019_2_0
Extracted 2115091_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2115091_31019_2_0
Extracted 2487290_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/2487290_31019_2_0
Extracted 3977315_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/3977315_31019_2_0
Extracted 5180744_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5180744_31019_2_0
Extracted 5180361_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/5180361_31019_2_0
Extracted 4913571_31019_2_0.zip to /Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB/4913571_31019_2_0
Extracted 3437578_31019_2_0.zip to /Users/xiao

In [19]:
#transform to BIDS format
import os
import re
import shutil
# Function to rename files to BIDS format
#session: Instance(2=Imaging visit; 3=First repeat imaging visit)
def rename_to_bids(base_dir,target_dir):
    # Loop through each subdirectory in the base directory
    for subject_dir in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_dir)

        if os.path.isdir(subject_path):
            # Extract the subject ID from the folder name
            subject_id = re.split(r'[_\s;]+', subject_dir)[0]
            session_id =re.split(r'[_\s;]+', subject_dir)[2]
        
            # Loop through the files in the subject directory
            for filename in os.listdir(subject_path):
                if filename.endswith('.csv.gz'):
                    space_id = re.split(r'[.\s;]+', filename)[1]
                    #sub-subID_ses-mri_run-runID_task-{task_name}_space-space.csv.gz
                    task_name = "rest"  # Modify this if you have different tasks
                    new_filename = f"sub-{subject_id}_ses-{session_id}_task-{task_name}_space-{space_id}.csv.gz"
                    new_file_path = os.path.join(target_dir, new_filename)

                    # Full path of the old file
                    old_file_path = os.path.join(subject_path, filename)

                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    #shutil.copy(old_file_path, new_file_path)
                    #print(f"Renamed: {old_file_path} to {new_file_path}")

# Set the path to your directory containing the files
# base_dir_parent = "/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB"
# dict_data_sets = get_all_folders(base_dir_parent)
# for folder in dict_data_sets.keys():
#     base_dir = dict_data_sets[folder]
#     target_dir = os.path.join("/Users/xiaoqianxiao/UKB/data/derivatives/timeseries",folder)
#     if not os.path.exists(target_dir):
#         os.makedirs(target_dir)
#     # Call the function
#     rename_to_bids(base_dir, target_dir)

In [86]:
def rename_to_bids(base_dir,target_dir):
    for subject_dir in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_dir)
        if os.path.isdir(subject_path):
            # Extract the subject ID from the folder name
            subject_id = re.split(r'[_\s;]+', subject_dir)[0]
            session_id =re.split(r'[_\s;]+', subject_dir)[2]
        
            # Loop through the files in the subject directory
            for filename in os.listdir(subject_path):
                if filename.endswith('.csv.gz'):
                    space_id = re.split(r'[.\s;]+', filename)[1]
                    #sub-subID_ses-mri_run-runID_task-{task_name}_space-space.csv.gz
                    task_name = "rest"  # Modify this if you have different tasks
                    new_filename = f"sub-{subject_id}_ses-{session_id}_task-{task_name}_space-{space_id}.csv.gz"
                    new_file_path = os.path.join(target_dir, new_filename)

                    # Full path of the old file
                    old_file_path = os.path.join(subject_path, filename)

                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    #shutil.copy(old_file_path, new_file_path)
                    #print(f"Renamed: {old_file_path} to {new_file_path}")
base_dir = '/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB'
target_dir = '/Users/xiaoqianxiao/UKB/data/derivatives/timeseries'
rename_to_bids(base_dir, target_dir)